# Getting Data From Instagram

In this project, I am going to use selenium webdriver to get data from instagram and then submit it to google sheet using a libary call pygsheets

Import all libarys

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import urllib, os, urllib.request
import time
import datetime
import zipfile
from os import listdir
from os.path import isfile, join
import zipfile
import pygsheets
import pandas as pd
import numpy as np
import json
import math
import re, json, requests
import random

The first two function are: 1. accessing the google sheet in my drive and getting all records from it. I need to get a list of IG user so that later on will put them to a function. 2. After finish getting all the data, I will put it back to the original drive.

Before running, you need to:
1. Set up Google Sheet API,
2. Download the service account file (key) and put it to an identifiable directory.
3. Share the sheet to the service account email.

In [ ]:
def sheet_access():
    
    gc = pygsheets.authorize(service_file='key.json')
    ss = gc.open_by_url('https://docs.google.com/spreadsheets/d/1e_y4AS7Rqjdm_pql6KjvUZuod96YZ2FVr6Dc5PwO5mE/edit#gid=0')
    ws = ss.worksheet('title','Demand Analysis')
    data = ws.get_all_records()
    df = pd.DataFrame(data)
    
    return df

def set_table(df):
    
    gc = pygsheets.authorize(service_file='key.json')
    ss = gc.open_by_url('https://docs.google.com/spreadsheets/d/1e_y4AS7Rqjdm_pql6KjvUZuod96YZ2FVr6Dc5PwO5mE/edit#gid=0')
    ws = ss.worksheet('title','Demand Analysis')
    ws.set_dataframe(df=df,start="A1")
    
def initialize():
                               
    driver = webdriver.Chrome()
    driver.get("https://www.instagram.com/accounts/login/")
    time.sleep(2.5)
    username = driver.find_elements_by_tag_name('input')[0]
    password = driver.find_elements_by_tag_name('input')[1]

    username.send_keys("a_helloshop")
    password.send_keys("helloshop")

    driver.find_elements_by_tag_name("button")[1].click()
    time.sleep(2.5)
    return driver

def get_json_from_sel(end_cursor="",driver=None,shopname=None):
    
    driver.get("https://www.instagram.com/"+shopname+"/?__a=1&max_id={0}".format(end_cursor))
    time.sleep(2)
    data = driver.find_elements_by_tag_name('pre')[0].text
    
    return json.loads(data)

First, Initialize the driver

In [ ]:
mydriver = initialize()
end_cursor = ''

It will pop up a Chrome Browser where specify software testing. Then Access to the google sheets.

In [2]:
df = sheet_access()

The format is like below. Two cols with urls for each users. For me, I am trying to observer some IG store sellers. So I use  sellers as their column name.

In [3]:
df.head()

,Seller Name,Sellers URL
0,just_bid_it2,https://www.instagram.com/just_bid_it2/
1,nathan_sneaker,https://www.instagram.com/nathan_sneaker/
2,dd.sneakers,https://www.instagram.com/dd.sneakers/
3,toponline.shop,https://www.instagram.com/toponline.shop/
4,lookrightt,https://www.instagram.com/lookrightt/


First Function is trying to run all the functions belows, where each of them are trying to send a request to a shop url and grap a json back. Then pull the data out.

In [6]:
def update(df,*args):
    shopname_list = df.iloc[:,0].to_list()
    find_followers(shopname_list,df,mydriver)
    find_posts_num(shopname_list,df,mydriver)
    find_following(shopname_list,df,mydriver)
    find_recent_12_likes(shopname_list,df,mydriver)
    find_bio(shopname_list,df,mydriver)
    find_all_post(shopname_list,df,mydriver)
    
def find_followers(shopname_list,df,mydriver):
    lists = []
    for shop in shopname_list:
        data = get_json_from_sel(end_cursor = end_cursor, driver = mydriver ,shopname = shop)
        lists.append(data['graphql']['user']['edge_followed_by']['count'])
    df['followers'] = lists   
    
def find_posts_num(shopname_list,df,mydriver):
    lists = []
    for shop in shopname_list:
        data = get_json_from_sel(end_cursor = end_cursor, driver = mydriver ,shopname = shop)
        lists.append(data['graphql']['user']['edge_owner_to_timeline_media']['count'])
    df['posts_num'] = lists   

def find_following(shopname_list,df,mydriver):
    lists = []
    for shop in shopname_list:
        data = get_json_from_sel(end_cursor = end_cursor, driver = mydriver ,shopname = shop)
        lists.append(data['graphql']['user']['edge_follow']['count'])
    df['following'] = lists   

def find_recent_12_likes(shopname_list,df,mydriver):
    lists = []
    for shop in shopname_list:
        data = get_json_from_sel(end_cursor = end_cursor, driver = mydriver ,shopname = shop)
        likes_list = []
        for i in range(len(data['graphql']['user']['edge_owner_to_timeline_media']['edges'])):
            likes_list.append(data['graphql']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_liked_by']['count'])
        lists.append(np.mean(likes_list))
    df['avg_12_posts_likes'] = lists   

    
def find_bio(shopname_list,df,mydriver):
    lists = []
    for shop in shopname_list:
        data = get_json_from_sel(end_cursor = end_cursor, driver = mydriver ,shopname = shop)
        lists.append(data['graphql']['user']['biography'])
    df['bio'] = lists   

def find_all_post(shopname_list,df,mydriver):
    end_cursor = ''
    lists = []
    for shop in shopname_list:
        data = get_json_from_sel(end_cursor = end_cursor, driver = mydriver ,shopname = shop)
        length = data['graphql']['user']['edge_owner_to_timeline_media']['count']
    
        nodes = []
        
        page_count = math.ceil(length/12) # desired number of pages
        for i in range(0, page_count):
            data = get_json_from_sel(end_cursor = end_cursor, driver = mydriver ,shopname = shop)

            end_cursor = data['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'] # value for the next page
            edges = data['graphql']['user']['edge_owner_to_timeline_media']['edges'] # list with posts

            for item in edges:
                nodes.append(item['node'])
            print(shop, "Current Post: ", len(nodes), " / ", str(length))
            time.sleep(1) # insurence to not reach a time limit
            
        lists.append(nodes)
    df['allpost'] = lists

In [7]:
update(df, find_followers, find_posts_num, find_following, find_recent_12_likes, find_bio, find_all_post)

just_bid_it2 Current Post:  12  /  237
just_bid_it2 Current Post:  24  /  237
just_bid_it2 Current Post:  36  /  237
just_bid_it2 Current Post:  48  /  237
just_bid_it2 Current Post:  60  /  237
just_bid_it2 Current Post:  72  /  237
just_bid_it2 Current Post:  84  /  237
just_bid_it2 Current Post:  96  /  237
just_bid_it2 Current Post:  108  /  237
just_bid_it2 Current Post:  120  /  237
just_bid_it2 Current Post:  132  /  237
just_bid_it2 Current Post:  144  /  237
just_bid_it2 Current Post:  156  /  237
just_bid_it2 Current Post:  168  /  237
just_bid_it2 Current Post:  180  /  237
just_bid_it2 Current Post:  192  /  237
just_bid_it2 Current Post:  204  /  237
just_bid_it2 Current Post:  216  /  237
just_bid_it2 Current Post:  228  /  237
just_bid_it2 Current Post:  240  /  237
just_bid_it2 Current Post:  252  /  237
just_bid_it2 Current Post:  264  /  237
just_bid_it2 Current Post:  276  /  237
just_bid_it2 Current Post:  288  /  237
just_bid_it2 Current Post:  300  /  237
just_bid

just_bid_it2 Current Post:  2448  /  237
just_bid_it2 Current Post:  2460  /  237
just_bid_it2 Current Post:  2472  /  237
just_bid_it2 Current Post:  2484  /  237
just_bid_it2 Current Post:  2496  /  237
just_bid_it2 Current Post:  2508  /  237
just_bid_it2 Current Post:  2520  /  237
just_bid_it2 Current Post:  2532  /  237
just_bid_it2 Current Post:  2544  /  237
just_bid_it2 Current Post:  2556  /  237
just_bid_it2 Current Post:  2568  /  237
just_bid_it2 Current Post:  2580  /  237
just_bid_it2 Current Post:  2592  /  237
just_bid_it2 Current Post:  2604  /  237
just_bid_it2 Current Post:  2616  /  237
just_bid_it2 Current Post:  2628  /  237
just_bid_it2 Current Post:  2640  /  237
just_bid_it2 Current Post:  2652  /  237
just_bid_it2 Current Post:  2664  /  237
just_bid_it2 Current Post:  2676  /  237
just_bid_it2 Current Post:  2688  /  237
just_bid_it2 Current Post:  2700  /  237
just_bid_it2 Current Post:  2712  /  237
just_bid_it2 Current Post:  2724  /  237
just_bid_it2 Cur

nathan_sneaker Current Post:  1944  /  243
nathan_sneaker Current Post:  1956  /  243
nathan_sneaker Current Post:  1968  /  243
nathan_sneaker Current Post:  1980  /  243
nathan_sneaker Current Post:  1992  /  243


IndexError: list index out of range

After Running the cell, use set_table() function pass in the new dataframe (df) to load the data back to google sheet.

In [9]:
set_table(df)

In [8]:
display(df)

,Seller Name,Sellers URL,followers,posts_num,following,avg_12_posts_likes,bio
0,just_bid_it2,https://www.instagram.com/just_bid_it2/,56135,2836,7217,422.166667,Since2014\n🈴正版正貨編號5938\n旺角門市每天營業🕑14:00-21:00\n...
1,nathan_sneaker,https://www.instagram.com/nathan_sneaker/,34719,2913,7023,49.416667,N⃞A⃞T⃞H⃞A⃞N⃞ S⃞N⃞E⃞A⃞K⃞E⃞R⃞📣\nHK-90後創業Nąţhąɲ\n...
2,dd.sneakers,https://www.instagram.com/dd.sneakers/,62340,7020,7457,183.583333,波鞋訂購/現貨/潮物 多國直送 \nwtsapp: 9074 5799\n📪 免郵費(順豐自...
3,toponline.shop,https://www.instagram.com/toponline.shop/,829,225,2998,13.583333,-👍高質素物料生產💯信心保證❤️\n-\n📋落單請Cut圖聯絡我😊\n-\n📲WhatsAp...
4,lookrightt,https://www.instagram.com/lookrightt/,79287,7455,3808,196.666667,🔥百老匯戲院斜對面 麥當奴旁\n🔥20樓出電梯右手第一間\n🎄20-29/12 3-9PM\...
5,grocery_store_852,https://www.instagram.com/grocery_store_852/,18727,3856,2,125.416667,Grocery store 852 \nPOP up start 1/1 🔥🔥🔥\nWelc...
6,superb_mart_,https://www.instagram.com/superb_mart_/,27972,884,5463,179.333333,開店超過3年賣出過萬件衫\n學生哥店主👨🏻‍🎓專買抵玩嘅衫👻\n日韓Dickies Cham...
7,loveycrazy,https://www.instagram.com/loveycrazy/,54811,2809,278,293.583333,預訂優惠: @loveycrazy_ \n \n🏠門市地址：旺角好景商業中心7樓704室\n...
8,sup3store,https://www.instagram.com/sup3store/,39053,890,1,582.750000,👧🏻店主=店員=客服=MODEL\n📍香港•澳門•台灣配送🌎✈️\n💥CHAMPION$68...
9,kikilittlestore,https://www.instagram.com/kikilittlestore/,212,59,117,15.250000,HELLO 我係04‘s 既小店主\n不接急單 ！！\n除咗迪士尼同埋K Pop嘢之外 都會...
